In [1]:
# Build requests modules
import pandas as pd
import requests
import json
import asyncio
import aiohttp
import datetime
import time
import json
import numpy as np
import os
import pandas as pd
from pandas.io.json import json_normalize
import plotly
import redis
from multiprocessing.pool import ThreadPool
import asyncio
import aiohttp
import time
import urllib.request
from sqlalchemy import create_engine, text
import plotly.express as px

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
class nhlstats():

    def __init__(self):
        self.url = "https://statsapi.web.nhl.com/api/v1/"

    def player(self, player_id):
        url = self.url + "people" + "/{}".format(self.player_id)
        r = requests.get(url)
        js = r.json()
        df = pd.json_normalize(js)
        return df
    
    def playerSeasonStats(self, player_id, season):
        url = self.url + "people/{}/stats?stats=statsSingleSeason&season={}".format(player_id, season)
        r = requests.get(url)
        js = r.json()
        df = pd.json_normalize(js["stats"], record_path=["splits"])
        return df
    
    def seasons(self):
        url = self.url + "seasons"
        r = requests.get(url)
        js = r.json()
        df = pd.json_normalize(js["seasons"])
        return df

    def standings(self):
        url = self.url + "standings"
        js = requests.get(url).json()
        df = pd.json_normalize(js["records"], record_path=["teamRecords"])
        return df

    # IF YOU CAN COMBINE TEAMS INTO ONE FUNCTION WITH SUBFUNCTIONS
    def teamsList(self):
        url = self.url + "teams"
        js = requests.get(url).json()
        df = pd.json_normalize(js["teams"])
        return df

    def teamStats(self, team_id=None, season=None):

        if team_id is not None:
            url = self.url + "teams" + "/{}".format(team_id)
        else:
            url = self.url + "teams"

        try:
            if season is not None:
                r = requests.get(url + "?expand=team.stats" + "&season={}".format(season))
            else:
                #current season
                r = requests.get(url + "?expand=team.stats")
            js = r.json()
            df = pd.json_normalize(data=js["teams"], record_path=["teamStats", "splits"])
            return df
        except:
            df = pd.DataFrame()
            return df

    def teamRoster(self, team_id=None, season=None):

        if team_id is not None:
            url = self.url + "teams" + "/{}".format(team_id)
        else:
            url = self.url + "teams"

        if season is not None:
            r = requests.get(url + "?expand=team.roster" + "&season={}".format(season))
        else:
            #current season
            r = requests.get(url + "?expand=team.roster")
        js = r.json()
        df = pd.json_normalize(data=js["teams"], record_path=["roster", "roster"], meta=["id", "name"])
        return df

    def teamLogo(team_id):
        if team_id is None:
            return 'Error: team id required'
        else:
            url = 'https://www-league.nhlstatic.com/images/logos/teams-current-primary-light/' + str(team_id)  + '.svg'
            return url

In [41]:
playerids = df_players['person.id'].unique()
seasons = df_players['season'].unique()

def build_playerstats(player, season):
    df_players = pd.DataFrame()
    df = nhlstats().playerSeasonStats(player, season)
    df['season'] = season
    df['person.id'] = player
    df_players = df_players.append(df)
    return df_players
# Use threader
def run_downloader_playerlist(process:int, playerids:list, season:str):
    print(process)
    len_playerids = len(playerids)
    df_players = pd.DataFrame()
    season_playerids = [season]*len_playerids
    results = ThreadPool(process).starmap(build_playerstats, zip(playerids, season_playerids))
    for r in results:
        df_players = df_players.append(r)
    return df_players
try:
    num_process = int(sys.argv[2])
except:
    num_process = 100
df_plyr_stats_ten = pd.DataFrame()   
for season in seasons:
    print(season)
    df_plyr_stats = run_downloader_playerlist(num_process, playerids, season)
    df_plyr_stats_ten = df_plyr_stats_ten.append(df_plyr_stats)
#df_players.to_sql('players', con=engine, if_exists='replace')

20122013
100
20132014
100
20142015
100
20152016
100
20162017
100
20172018
100
20182019
100
20192020
100
20202021
100
20212022
100


In [49]:
df_plyr_stats_ten.to_sql('playersTenStats', con=engine, if_exists='replace')

9950

In [45]:
df_plyr_stats_ten.columns

Index(['season', 'stat.timeOnIce', 'stat.assists', 'stat.goals', 'stat.pim',
       'stat.shots', 'stat.games', 'stat.hits', 'stat.powerPlayGoals',
       'stat.powerPlayPoints', 'stat.powerPlayTimeOnIce', 'stat.evenTimeOnIce',
       'stat.penaltyMinutes', 'stat.faceOffPct', 'stat.shotPct',
       'stat.gameWinningGoals', 'stat.overTimeGoals', 'stat.shortHandedGoals',
       'stat.shortHandedPoints', 'stat.shortHandedTimeOnIce', 'stat.blocked',
       'stat.plusMinus', 'stat.points', 'stat.shifts', 'stat.timeOnIcePerGame',
       'stat.evenTimeOnIcePerGame', 'stat.shortHandedTimeOnIcePerGame',
       'stat.powerPlayTimeOnIcePerGame', 'person.id', 'stat.ot',
       'stat.shutouts', 'stat.wins', 'stat.losses', 'stat.saves',
       'stat.powerPlaySaves', 'stat.shortHandedSaves', 'stat.evenSaves',
       'stat.shortHandedShots', 'stat.evenShots', 'stat.powerPlayShots',
       'stat.savePercentage', 'stat.goalAgainstAverage', 'stat.gamesStarted',
       'stat.shotsAgainst', 'stat.goalsAgai

In [35]:
df_plyr_stats_ten.columns

Index(['season', 'stat.timeOnIce', 'stat.assists', 'stat.goals', 'stat.pim',
       'stat.shots', 'stat.games', 'stat.hits', 'stat.powerPlayGoals',
       'stat.powerPlayPoints', 'stat.powerPlayTimeOnIce', 'stat.evenTimeOnIce',
       'stat.penaltyMinutes', 'stat.faceOffPct', 'stat.shotPct',
       'stat.gameWinningGoals', 'stat.overTimeGoals', 'stat.shortHandedGoals',
       'stat.shortHandedPoints', 'stat.shortHandedTimeOnIce', 'stat.blocked',
       'stat.plusMinus', 'stat.points', 'stat.shifts', 'stat.timeOnIcePerGame',
       'stat.evenTimeOnIcePerGame', 'stat.shortHandedTimeOnIcePerGame',
       'stat.powerPlayTimeOnIcePerGame', 'person.id', 'stat.ot',
       'stat.shutouts', 'stat.wins', 'stat.losses', 'stat.saves',
       'stat.powerPlaySaves', 'stat.shortHandedSaves', 'stat.evenSaves',
       'stat.shortHandedShots', 'stat.evenShots', 'stat.powerPlayShots',
       'stat.savePercentage', 'stat.goalAgainstAverage', 'stat.gamesStarted',
       'stat.shotsAgainst', 'stat.goalsAgai